<a href="https://colab.research.google.com/github/francismwendwa22/computer_vision/blob/main/yolov5_training_with_roboflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#clone YOLOv5 and 
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
%pip install -qr requirements.txt # install dependencies
%pip install -q roboflow

import os
import matplotlib.pyplot as plt
import cv2
import requests
import random
import numpy as np
import torch

SEED = 42
np.random.seed(SEED)

TRAIN = True

from IPython.display import Image, clear_output  # to display images

print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Cloning into 'yolov5'...
remote: Enumerating objects: 15002, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 15002 (delta 1), reused 3 (delta 1), pack-reused 14995
Receiving objects: 100% (15002/15002), 14.03 MiB | 16.53 MiB/s, done.
Resolving deltas: 100% (10287/10287), done.
/content/yolov5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.0/184.0 KB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 65.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 KB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 KB 15.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 KB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 KB 8.9 MB/s eta 0:00:00
Setup complete. Using torch 1.13.1+cu116

In [ ]:
from roboflow import Roboflow
rf = Roboflow(model_format="yolov5", notebook="ultralytics")

In [ ]:
# set up environment
os.environ["DATASET_DIRECTORY"] = "/content/datasets"

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="DItxO3lhPvqgRFXGmcFO")
project = rf.workspace("aico").project("drone-detection-jwzcv")
dataset = project.version(1).download("yolov5")

In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs

In [ ]:
def set_res_dir():
        
        res_dir_count = len(glob.glob('runs/train/*'))
        print(f"Current number of result directories: {res_dir_count}")
        if TRAIN:
            RES_DIR = f"results_{res_dir_count+1}"
            print(RES_DIR)
        else:
            RES_DIR = f"results_{res_dir_count}"
        return RES_DIR

In [ ]:
import glob as glob
RES_DIR = set_res_dir()
if TRAIN:
    !python train.py --img 416 --batch 16 --epochs 40 --data {dataset.location}/data.yaml --weights yolov5s.pt --cache

In [ ]:
!python detect.py --weights runs/train/exp/weights/best.pt --img 416 --conf 0.1 --source {dataset.location}/test/images

In [ ]:
#display inference on ALL test images

import glob
from IPython.display import Image, display

for imageName in glob.glob('/content/yolov5/runs/detect/exp/*.jpg'): #assuming JPG
    display(Image(filename=imageName))
    print("\n")

In [ ]:
#export your model's weights for future use
from google.colab import files
files.download('./runs/train/exp/weights/best.pt')